In [286]:
struct HMM
    K::Int # number of states
    D::Int # observation size of emission
    π::Array{Float64, 1} # initial state probability matrix, Kx1
    T::Array{Float64, 2} # transition probability matrix, Kx(K+1)   
    E::Array{Float64, 2} # emission probability matrix, KxD
    function HMM(K::Int, D::Int)
        π  = rand(K)
        π ./= sum(π)
        E = rand(K,D)
        E = E ./ sum(E,dims=2)
        T = rand(K, K)
        T ./= sum(T,dims=2)
        new(K, D,π,T, E)
    end        
end

function emission(hmm::HMM, s::Int)
    e = zeros(hmm.D)
    #println(hmm.E[s,:])
    cs = cumsum(hmm.E[s,:],dims=1)
    #println(cs)
    rnd = rand()
    #println(rnd)
    dims = cs .<= rnd
    #println(dims)
    e[sum(dims) + 1] = 1
    return e
end

function generate(hmm::HMM, t::Int)
    #println(hmm.π)
    #println(cumsum(hmm.π, dims=1))
    rnd = rand()
    #println(rnd)
    cs = cumsum(hmm.π, dims=1)
    #println(cs)
    #println(cs .<= rnd)
    state = sum(cs .<= rnd) + 1
    #println("Starting state $state")
    #return
    #println(hmm.T)
    #println(hmm.T[1,:])
    observations = zeros(hmm.D, t)
    for i=1:t
        o = emission(hmm, state)
        observations[:,i] = o
        #println(hmm.T[state,:])
        rnd = rand()
        #println(rnd)
        #println(cumsum(hmm.T[state,:], dims=1))
        #println(sum(cumsum(hmm.T[state,:], dims=1) .<= rnd) )
        state = sum(cumsum(hmm.T[state,:], dims=1) .<= rnd) + 1
        #println("new state $state")
    end
    observations
    #return hmm.π * hmm.E
end

#X = [ 1 0 1 0; 0 1 0 0; 0 0 0 1 ]
#X

generate (generic function with 1 method)

In [298]:
# estep
function e_step(hmm::HMM, X::Array{Float64,2})
    timesteps = size(X)[2]
    # set up alpha matrix
    alpha = zeros(hmm.K,timesteps)
    # set first column to the emission probability given state of the first observation, 
    # multiplied by probability of starting in that state
    alpha[:,1] = (hmm.π .* (hmm.E * X))[:,1]

    for t=2:size(X)[2]
        for s=1:hmm.K
            for s_=1:hmm.K
                if s_ != s
                    alpha[s,t] = alpha[s,t-1] * hmm.T[s,s_] * sum(hmm.E[s] * X[:,t])
                end
            end
        end
    end

    f_prob = alpha[lastindex(alpha)] + alpha[lastindex(alpha)-1]
    f_prob # P of the observation (from t=1 to T=T) given current model parameters
    
    # set up beta matrix
    beta = zeros(hmm.K,timesteps)
    # set last column of beta matrix to 1
    beta[:,timesteps] .= 1
    E_X = hmm.E * X

    for t in timesteps-1:-1:1
        for s_i in 1:hmm.K
            accum = 0
            for s_j in 1:hmm.K
                accum += alpha[s_i,s_j] * E_X[s_j,t+1] * beta[s_j,t+1]
            end
            beta[s_i,t] = accum
        end
    end
    return alpha, beta, f_prob
end

e_step (generic function with 2 methods)

In [305]:
function m_step(hmm::HMM, X::Array{Float64,2}, alpha::Array{Float64,2}, beta::Array{Float64,2}, f_prob::Float64)
    timesteps = size(X)[2]
    E_X = hmm.E * X

    # gamma is Kxtimesteps
    gamma = alpha .* beta ./ f_prob
    ksi=zeros(hmm.K,hmm.K,timesteps-1)
    for t=1:timesteps-1
        for s_i=1:hmm.K
            for s_j=1:hmm.K
                ksi[s_i,s_j,t] = (alpha[s_i,t] * hmm.T[s_i, s_j] * E_X[t+1] * beta[s_j, t+1]) / f_prob
            end
        end
    end
    for s_i=1:hmm.K
        for s_j=1:hmm.K
            for t=1:timesteps
                #println(ksi[s_i,s_j,1:timesteps-1]) / sum(ksi[s_i,:,:]))
                hmm.T[s_i, s_j] = sum(ksi[s_i,s_j,1:timesteps-1]) / sum(ksi[s_i,:,:])
            end
        end
    end

    for s_j=1:hmm.K
        total = sum(gamma[s_j,:])

        for v=1:hmm.D
            accum = 0
            for t=1:timesteps
               if(v==argmax(X[:,t]))
                    accum += gamma[s_j,t]
                end 
            end
            hmm.E[s_j,v] = accum / total
        end
    end
end

m_step (generic function with 3 methods)

In [316]:
function train(hmm::HMM, X::Array{Float64,2})
    for i in 1:10
        alpha, beta, f_prob = e_step(hmm,X)
        m_step(hmm,X, alpha, beta, f_prob)    
    end
end


train (generic function with 1 method)

In [325]:
hmm_actual = HMM(2, 3)
generated = generate(hmm_actual, 5)
print(generated)
println(size(generated))
#return
#hmm_model = HMM(2, 3)
#println(hmm_actual.E)
#println(hmm_model.E)
#train(hmm_model, generate(hmm_actual, 5))
#println(hmm_model.E)
#train(hmm_model, generate(hmm_actual, 5))
#println(hmm_model.E)
#train(hmm_model, generate(hmm_actual, 5))
#println(hmm_model.E)

[1.0 0.0 1.0 1.0 1.0; 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0](3, 5)


[0.2547050416454581 0.3824355909785932 0.3628593673759487; 0.14933481913013563 0.389344433972019 0.46132074689784536]
[0.080266620380813 0.9197333796191871 0.0; 1.0 0.0 0.0]
